# Creación de Dataset

Este notebook se utiliza para la creación del conjunto de datos para el modelo, primero muestreamos fotogramas con las manos en las posiciones deseadas, después, procesamos cada imágen y escribimos los datos en un archivo.

Por el momento se ha implementado de manera separado para no realizar procesos adicionales durante el muestreo, pero si demuestra no ser de mucho impacto, se podría implementar como un solo código.

In [ ]:
# Instalar en caso de no tenerlo
!pip install mediapipe
!pip install opencv-python

In [2]:
# Dependencias
import os
import cv2
import pickle # Pickle ya es parte de Python en versiones mas nuevas :)
import mediapipe as mp
import matplotlib.pyplot as plt

## Creacion de imagenes
En esta sección, tomamos un video con las muestras del gesto que queremos muestrear

In [3]:
# Definimos el path donde de van a guardar todas las fotos
DATA_DIR = './model/data'
if not os.path.exists(DATA_DIR):
    os.makedirs(DATA_DIR)

In [4]:
def collectData(class_number, dataset_size):
    # Definimos el directorio, que fue creado en la celda anterior
    DATA_DIR = './model/data'

    # Inicializamos el metodo de captura de webcam, el parámetro corresponde para el número de dispositivo, tengo entendido que para WINDOWS es 0 y para MAC es 2
    cap = cv2.VideoCapture(0)

    # Creamos un directorio para la clase elegida si este no existe ya
    class_dir = os.path.join(DATA_DIR, str(class_number))
    if not os.path.exists(class_dir):
        os.makedirs(class_dir)

    # Encontramos el ultimo archivo guardado dentro del directorio, para guardarlo bajo ese nombre, ej: 1.png
    existing_images = os.listdir(class_dir)
    if existing_images:
        last_image_number = max([int(img.split('.')[0]) for img in existing_images])
    else:
        # Si no existe, empezamops a contar desde 0
        last_image_number = 0

    # Empezamos entonces a contar a partir del siguiente disponible
    counter = last_image_number + 1

    
    # Variable usada despues para salir del programa de manera manual
    done = False
    # Ciclo inicial para determinar si el usuario va a presionar Q o ESC
    while counter <= dataset_size + last_image_number:

        # Leemos cada cuadro
        ret, frame = cap.read()

        #Instrucciones en pantalla
        cv2.putText(frame, 'Collecting data for class {} - Press "Q" to start'.format(class_number),
                    (0, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2, cv2.LINE_AA)
        cv2.putText(frame, 'Press "Esc" to exit', (0, frame.shape[0] - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2, cv2.LINE_AA)
        cv2.imshow('frame', frame)

        key = cv2.waitKey(25)
        if key == ord('q'): # Confirmamos que queremos seguir con q, entonces rompemos el ciclo actual y vamos al siguiente
            break
        elif key == 27:  # Si se presiona esc, salimos del programa
            done = True
            break
    
    # Iniciamos este ciclo al hacer break del anterior y si done no es verdad
    # Ciclo para recolectar datos
    while counter <= dataset_size + last_image_number and not done:
        ret, frame = cap.read()
        cv2.imshow('frame', frame)
        # Espera input, necesario para que no se rompa el codigo
        cv2.waitKey(25)
        cv2.imwrite(os.path.join(class_dir, '{}.jpg'.format(counter)), frame)

        counter += 1

    # Liberamos recursos y mandamos mensaje de confirmacion
    cap.release()
    cv2.destroyAllWindows()
    print('Collecting for class {} successful'.format(class_number))
        

In [8]:
gesture = 1 
size = 100
collectData(gesture, size)

## Obtención de coordenadas a partir de las muestras

En esta parte, usamos mediapipe para obtener las coordenadas de manos de las imágenes anteriores y lo guardamos en un archivo para procesar en el modelo ML

In [10]:
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

hands = mp_hands.Hands(static_image_mode=True, min_detection_confidence=0.3)


In [11]:
# Definimos el directorio, que fue creado en la celda anterior
DATA_DIR = './model/data'

# Listas para guardar los datos de cada gesto y su label correspondiente
data = []
labels = []

# Por cada subdirectorio en DATA_DIR
for dir_ in os.listdir(DATA_DIR):
    # Por cada imagen en el subdirectorio actuial
    for img_path in os.listdir(os.path.join(DATA_DIR, dir_)):
        # Lista auxiliar para guardar los datos de cada mano al final
        data_aux = []

        # Lista para guardar las coordenadas X y Y de cada mano
        x_ = []
        y_ = []

        # Convertimos a RGB para procesar con MediaPipe
        img = cv2.imread(os.path.join(DATA_DIR, dir_, img_path))
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        # Procesamos la imagen con MediaPipe

        results = hands.process(img_rgb)


        # Si se detectaron manos en la imagen
        if results.multi_hand_landmarks:

            # Para cada mano detectada
            for hand_landmarks in results.multi_hand_landmarks:
                # Extraemos la coordenada en X y Y de esta y las guardamos en x_ y_
                for i in range(len(hand_landmarks.landmark)):
                    x = hand_landmarks.landmark[i].x
                    y = hand_landmarks.landmark[i].y

                    x_.append(x)
                    y_.append(y)

                # Normalizamos las medidas y las guardamos en data_aux

                for i in range(len(hand_landmarks.landmark)):
                    x = hand_landmarks.landmark[i].x
                    y = hand_landmarks.landmark[i].y
                    data_aux.append(x - min(x_))
                    data_aux.append(y - min(y_))

            # Guardamos los datos de data_aux en la lista data, y labels se obtiene a partir del nombre del directorio

            data.append(data_aux)
            labels.append(dir_)

# Guardamos los datos como un archivo tipo Pickle

f = open(os.path.join('model', 'data.pickle'), 'wb')
pickle.dump({'data': data, 'labels': labels}, f)
f.close()